## Preamble

In [ ]:
strain= 'rdv2'
process = 'wholepop'
programs = ['aas', 'ras', 'dta', 'atd'] # each a type of experiment
# dta is dark to active light's experiment dataset, atd is active to dark dataset

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

%matplotlib inline
plt.viridis()

data_path = "../cph8-ompr_data" # the next folder over

dfs = {} # the variable with ALL the data
for pt in programs:
    prog_path = os.path.join(data_path, pt)
    xl_path = os.path.join(prog_path, '{}_{}_{}_complete.xlsx'.format(strain,process,pt)) # the {} are replaced with items in format()
    dfs[pt] = pd.read_excel(xl_path)

## Organize data

In [ ]:
aas_sorted = dfs['aas'].sort_values(['Spectral intensity','Top Centroid (nm)'])
das_sorted = dfs['ras'].sort_values(['Spectral intensity','Top Centroid (nm)'])
X1 = np.reshape(aas_sorted['Top Centroid (nm)'], (12,24))
Y1 = np.reshape(aas_sorted['Spectral intensity'], (12,24))
Z1 = np.reshape(aas_sorted['FL1 Mean'], (12,24))

X2 = np.reshape(das_sorted['Top Centroid (nm)'], (4,24))
Y2 = np.reshape(das_sorted['Spectral intensity'], (4,24))
Z2 = np.reshape(das_sorted['FL1 Mean'], (4,24))

# these are some of the data columns used

Z=Z1
Z[-3:,0] = np.nan

## Fit model to data

In [ ]:
from scipy.interpolate import interp1d
from scipy.integrate import ode

C = np.reshape(aas_sorted['Top Centroid (nm)'], (12,24))
X1ss = np.reshape(aas_sorted['Spectral intensity'], (12,24))
Y1 = np.reshape(aas_sorted['FL1 Mean'], (12,24)) # the mean GFP fluorescence across cells
X2ss = np.reshape(das_sorted['Spectral intensity'], (4,24))
Y2 = np.reshape(das_sorted['FL1 Mean'], (4,24))

dta_sorted = dfs['dta'].sort_values(['Activating intensity', 'Data time point'])
X = np.reshape(dta_sorted['Data time point'], (8,24)) # when light is turned on
Y = np.reshape(dta_sorted['FL1 Mean'], (8,24))

# rest of the data columns used

act_intens = sorted(set(dfs['dta']['Activating intensity'].values))
act_keys = ['act_{}'.format(x) for x in act_intens]

dta = {}
for i,key in enumerate(act_keys):
    dta[key] = {
        'act0': act_intens[i], # light intensity
        't': X[i], # time point
        'gm': Y[i] # FL1 mean
    }

data = dta
keys_dta = data.keys()
intens_dta = [data[key]['act0'] for key in keys_dta]
intens_sorted_dta = sorted(intens_dta)
keys_sorted_dta = [y for x,y in sorted(zip(intens_dta, keys_dta))]


dta_sorted = dfs['atd'].sort_values(['Activating intensity', 'Data time point'])
X = np.reshape(dta_sorted['Data time point'], (8,24)) # when light is turned off
Y = np.reshape(dta_sorted['FL1 Mean'], (8,24))

act_intens = sorted(set(dfs['atd']['Activating intensity'].values))
act_keys = ['act_{}'.format(x) for x in act_intens]

atd = {}
for i,key in enumerate(act_keys):
    atd[key] = {
        'act0': act_intens[i], # I
        't': X[i], # time
        'gm': Y[i] # fluorescence
    }
    
mask = np.ones(24, dtype=bool)

data = atd
keys_atd = data.keys()
intens_atd = [data[key]['act0'] for key in keys_atd]
intens_sorted_atd = sorted(intens_atd)
keys_sorted_atd = [y for x,y in sorted(zip(intens_atd, keys_atd))]


def residual_ss(params, led, log=True, plot=False):
    pcs1 = [params['k1_{}'.format(i)].value for i in range(23)] # creating a bunch of k1&k2 values in the params variable
    pcs2 = [params['k2_{}'.format(i)].value for i in range(23)]
    k1 = pcs1[led] # ** suggests that each k1_n value is for an LED **
    k2 = pcs2[led]
    
    if led == 0:
        imax = 9 # one LED can't get as bright as the others (see density plot in paper)
    else:
        imax = 12
    
    Xd1 = X1ss[:imax,led] # X1ss is spectral intensity
    Yd1 = Y1[:imax,led]
    Ym1 = np.array([steadystate(params, x, 0, led1=led, led2=8,pcs1=pcs1,pcs2=pcs2) for x in Xd1])
    Ye1 = 0.1*Yd1
    
    das_inten = 1.25
    Xd2 = X2ss[:imax-8,led]
    Yd2 = Y2[:imax-8,led]
    Ym2 = np.array([steadystate(params, x,das_inten, led1=led, led2=14,pcs1=pcs1,pcs2=pcs2) for x in Xd2])
    Ye2 = 0.1*Yd2
    
    res1 = (Yd1 - Ym1) / Ye1 # yd1 is mean fl1 fluorescence
    res2 = (Yd2 - Ym2) / Ye2
    
    res = np.array(res1.tolist() + res2.tolist())
    
    if plot:
        Xs = np.logspace(np.log10(Xd1[0]), np.log10(Xd1[-1]), 100)
        Ys1 = [steadystate(params, x, 0, led1=led, led2=14,pcs1=pcs1,pcs2=pcs2) for x in Xs]
        Ys2 = [steadystate(params, x,das_inten, led1=led, led2=14,pcs1=pcs1,pcs2=pcs2) for x in Xs]
        
        plt.errorbar(Xd1, Yd1, yerr=Ye1, ms=0, fmt='.')
        plt.errorbar(Xd2, Yd2, yerr=Ye2, ms=0, fmt='.')
        plt.plot(Xs,Ys1,'-')
        plt.plot(Xs,Ys2,'-')
        plt.xscale('log')
        plt.yscale('log')
        plt.ylim([1e3,1.6e5])
        plt.show()
    
    if any(np.isnan(np.array(res))):
        print(led, res)
    
    return res


def steadystate(params, l1,l2,led1=0,led2=8, pcs1=None, pcs2=None): # 2  different LEDs used per well
    # finds steady-state gfp production rate
    p = params
    kg = p['kg'].value
    kdr = p['kdr'].value
    k = p['k'].value
    n = p['n'].value
    b = p['b'].value
    a = p['a'].value    
    
    def hill(x):
        return b + a * k**n / (k**n + x**n)
    
    def k1(l1i, l2i):
        return l1i * pcs1[led1] + l2i * pcs1[led2] # light 1 and light 2 intensities multiplied by the unit k values for their LED
        # NOTE: THEY USED 2 LEDs PER WELL
        
    def k2(l1i, l2i):
        return l1i * pcs2[led1] + l2i * pcs2[led2]
    
    k1 = k1(l1, l2)
    k2 = k2(l1, l2)
   
    y = k1 / (k1 + k2 + kdr + kg)
    
    return hill(y/(1-y))

def modelf(params, i, run):
    if run == 'dta':
        data = dta
        key = keys_sorted_dta[i]
    elif run == 'atd':
        data = atd
        key = keys_sorted_atd[i]
    
    act0 = data[key]['act0']
    
    b = params['b'].value
    a = params['a'].value
    k = params['k'].value
    n = params['n'].value
    k1 = params['k1_14'].value # why the 14th LED?
    k2 = params['k2_14'].value
    kdr = params['kdr'].value
    kg = params['kg'].value
    tau = params['tau'].value
    
    
    c1_a = k1 / (k1 + k2)
    c1_k = (kg + kdr) / (k1 + k2)
    c1 = c1_a * act0 / (act0 + c1_k)
    if run == 'dta':
        c1i = 0
        c1f = c1
        c2 = (k1 + k2) * act0 + kg + kdr # ktot
    elif run == 'atd':
        c1i = c1
        c1f= 0
        c2 = kg + kdr
    
    def kp(z):
        result = []
        x = np.array(z)
        xlist = x.tolist()
        for x in xlist:
            if x < tau:
                result.append(c1i)
            else:
                result.append(c1f + (c1i-c1f) * np.exp(-c2*(x-tau)) )
        phy_fracs = result        
        
        kp_result = []
        for phy_frac in phy_fracs:
            if phy_frac >=1:
                kp_result.append(a)
            else:
                phy_ratio = phy_frac / (1 - phy_frac) # phy ratio is R, phy frac is y
                kp_result.append(b + a*k**n / (k**n + phy_ratio**n)) # hill
        return kp_result
    
    def gfp():
        
        def get_kp(tEnd):
            t_sim = np.linspace(0, tEnd, 10*tEnd + 1)
            yset_sim = kp(t_sim)
            return interp1d(t_sim,yset_sim)
        
        tEnd = 600
        yset_interp = get_kp(tEnd)
        
        def f(t,g):
            return kg * (yset_interp(t) - g[0])
        
        done = False
        while not done:
            try:
                y0 = [yset_interp(0)]
                t1 = 480
                dt = 1
                
                r = ode(f)
                r.set_integrator('vode',method='bdf',order=15,nsteps=3000)
                r.set_initial_value(y0, 0)
                
                t = [0]
                g = y0
                while r.successful() and r.t < t1:
                    r.integrate(r.t+dt)
                    t.append(r.t)
                    g.append(r.y[0])
                done = True
            except(ValueError):
                tEnd = 2*tEnd
                yset_interp = get_kp(tEnd)
                print("Extending tEnd to {} for {} {}".format(tEnd,run,i))
            
        return interp1d(t,g)
    
    return gfp()
    
def residual(params, plot=False): # error calculation between prediction (model) and data
    errs = []
    b = params['b'].value
    a = params['a'].value
    for runtype in ['dta','atd']:
        if runtype == 'dta':
            for i in i_set['dta']:
                key = keys_sorted_dta[i]
                Xd = np.copy(dta[key]['t'])

                Yd = dta[key]['gm']
                Ym = np.array(modelf(params, i, runtype)(Xd))
                Ye = 0.1*Yd
    
                if plot:
                    Xs = np.linspace(0,360,361)
                    Ys = np.array(modelf(params, i, runtype)(Xs))
                    
                    plt.errorbar(Xd,Yd,yerr=Ye)
                    plt.plot(Xs,Ys,'-')
                    plt.yscale('log')
                
                errs = errs + [(yd-ym)/ye for yd,ym,ye in zip(Yd,Ym,Ye)]
            if plot:
                plt.show()
            
        if runtype == 'atd':
            for i in i_set['atd']:
                key = keys_sorted_atd[i]
                Xd = np.copy(atd[key]['t'])
                                 
                Yd = atd[key]['gm']
                Ym = np.array(modelf(params, i, runtype)(Xd))
                Ye = 0.1*Yd
                                 
                if plot:
                    Xs = np.linspace(0,360,361)
                    Ys = np.array(modelf(params, i, runtype)(Xs))
                    
                    plt.errorbar(Xd,Yd,yerr=Ye)
                    plt.plot(Xs,Ys,'-')
                    plt.yscale('log')
                
                errs = errs + [(yd-ym)/ye for yd,ym,ye in zip(Yd,Ym,Ye)]
            if plot:
                plt.show()
                
    for i in range(23):
        for res in residual_ss(params, i, plot=plot):
            errs.append(res)
    
    print(np.sum(np.array(errs)**2))
    
    return np.array(errs)


import lmfit as lm
import pickle

params = lm.Parameters()

i_set = {}
i_set['dta'] = [0,1,2,3,4,5,6,7]
i_set['atd'] = [0,1,2,3,4,5,6,7]

k1s = [0.01 for i in range(13)]
k2s = [0.01 for i in range(13)]

k1s += [0.1 for i in range(13,23)]
k2s += [0.001 for i in range(13,23)]

# adding items to the parameters array

for i in range(23):
    params.add('k1_{}'.format(i), value = k1s[i], vary = True, min=1e-6)
    params.add('k2_{}'.format(i), value = k2s[i], vary = True, min=1e-6)

params.add('k', value = 1, vary=True, min=1e-6)

params.add('kdr', value = 0.1, vary = True)

params.add('tau', value = 5, vary=True, min=1e-6, max=20)
params.add('kg', value = 0.02, vary=True, min=1e-6)
params.add('n', value = 1, vary=True, min=1e-6)
params.add('b', value = 2e2, vary=True, min=1e-6)
params.add('a', value = 3.5e3, vary=True, min=1e-6)

k1holds = [19,20,21,22]
for i in k1holds: # these values won't be changed when reducing error (actually model fitting)
    params['k1_{}'.format(i)].vary = False
    params['k1_{}'.format(i)].value = 1e-6

k2holds = [11,12,13,14]
for i in k2holds:
    params['k2_{}'.format(i)].vary = False
    params['k2_{}'.format(i)].value = 1e-6

kp_model_fit = lm.minimize(residual, params, epsfcn=1e-4) # minimize just minimizes the first funtion by changing the parameters
# here, it is minimizing error (the residual function)
print(lm.fit_report(kp_model_fit))
nothing = residual(kp_model_fit.params, True)

fname = './fit_results/all_data_onetau_with_kholds_and_adjusted_initial'
kp_model_fit.params.dump(open('{}.params'.format(fname),'wb'))
pickle.dump(lm.fit_report(kp_model_fit),open('{}.p'.format(fname),'wb'))
